In [1]:
import csv
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('final_data.csv')


In [3]:
df=df.dropna()
df=df.set_index(['Unnamed: 0'])

In [4]:
df['follower_count']=df['follower_count']/1000000


In [5]:
df['Positive']=df['Positive']*1000
df['Negative']=df['Negative']*1000
df['Anticipation']=df['Anticipation']*1000
df['Disgust']=df['Disgust']*1000
df['Fear']=df['Fear']*1000
df['Joy']=df['Joy']*1000
df['Sadness']=df['Sadness']*1000
df['Surprise']=df['Surprise']*1000
df['Trust']=df['Trust']*1000

In [ ]:
pd.options.display.max_columns=11
df.describe()